CART Alteração em massa comprimento, largura e area executada

In [1]:
from openpyxl import load_workbook
filename = "ALTERACOES OCORRENCIAS(1).xlsm"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

<Worksheet "Planilha3">

In [4]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

{'DADOS  DIÁRIO DE OBRA': 'Serial RDO',
 None: 'Impacto',
 'DADOS OCORRÊNCIAS': 'Origem'}

In [32]:
from tqdm.notebook import tqdm
from datetime import datetime

rep_edit = []
company = Company.objects.get(name='CART')

try:
    for index, a in enumerate(tqdm(values)):
    
        if str(a['Serial'])[-4:] == '2021':
            number = int(str(a['Serial'])[:-4])
            number = '{:05}'.format(number) + '.2021'    
        elif str(a['Serial RDO'])[-4:] == '2022':
            number = int(str(a['Serial RDO'])[:-4])
            number = '{:05}'.format(number) + '.2022'
        if str(a['Serial'])[-4:] == '2023':
            numero_str = str(a['Serial']).rstrip('.')[:-4]
            numero = int(numero_str)
            numero = '{:05}'.format(numero) + '.2023'
        elif str(a['Serial'])[-4:] == '2024':
            numero_str = str(a['Serial']).replace('.', '')
            numero = int(numero_str)
            numero = '{:05}'.format(numero) + '.2024'
        else:
            numero = a['Serial']

        numero = a['Serial']

        rep = Reporting.objects.get(number=numero, company=company)
        rep.history_user = User.objects.get(username='rlcs')

        if a['Cód. ARTESP'] is not None:
            rep.form_data['artesp_code'] = a['Cód. ARTESP']
        # if a['Descrição'] is not None:
        #     rep.form_data['description'] = a['Descrição']
        # if a['Peso Executado'] is not None:
        #     rep.form_data['executed_weight'] = a['Peso Executado']

        rep.save()
        rep_edit.append(rep)
        rep_edit[0].__dict__

except Exception as e:
    print(f"Erro: {e}, linha da planilha: {index+2}")


  0%|          | 0/382 [00:00<?, ?it/s]

Searching KM on direction...
Found KM on Road 869: SP-327
Searching KM on direction...
Found KM on Road 838: SP-327
Searching KM on direction...
Found KM on Road 838: SP-327
Searching KM on direction...
Found KM on Road 816: SPA 424/270
Searching KM on direction...
Found KM on Road 816: SPA 424/270
Searching KM on direction...
Searching KM without direction...
Found KM on Road 816: SPA 424/270
Searching KM on direction...
Searching KM without direction...
Found KM on Road 816: SPA 424/270
Searching KM on direction...
Found KM on Road 838: SP-327
Searching KM on direction...
Found KM on Road 838: SP-327
Searching KM on direction...
Searching KM without direction...
Found KM on Road 816: SPA 424/270
Searching KM on direction...
Found KM on Road 869: SP-327
Searching KM on direction...
Found KM on Road 869: SP-327
Searching KM on direction...
Found KM on Road 869: SP-327
Searching KM on direction...
Found KM on Road 838: SP-327
Searching KM on direction...
Found KM on Road 838: SP-327
Sea

In [ ]:
user=User.objects.get(username="rlcs")
user

In [ ]:
from helpers.histories import bulk_update_with_history
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=800,user=user)

In [ ]:
company=Company.objects.get(name='Arteris Via Paulista - SISOBRAS/SISQUALI')
up=UserPermission.objects.filter(name='Suporte',companies=company)
company,up

In [ ]:
for a in up:
    a.companies.remove(company)
    uperm = a
    uperm.uuid=None
    uperm.companies.set([company])
    uperm.save()
    UserInCompany.objects.filter(company=company,permissions__name=a.name).update(permissions=uperm)

In [ ]:
rep=Reporting.objects.get(uuid='e1817019-5ed7-41dc-99b5-270d8b31aa99')

In [ ]:
rep.__dict__

In [ ]:
reps=Reporting.objects.filter(number__in=['111211.2023','111210.2023','111368.2023'])

In [ ]:
for a in reps:
    a.occurrence_type = OccurrenceType.objects.get(name='Reposição/Substituição (baixa refletância/furto/avariada) (solo/aérea)',
                                                   company__name='CART')
    a.save()

In [ ]:
company=Company.objects.get(name='Eixo Sp - Pavimento')

In [ ]:
reps=Reporting.objects.filter(occurrence_type__occurrence_kind__in=[5,6,8],company=company
                             ).exclude(form_data__has_key='service')

In [ ]:
reps.count()

In [ ]:
reps=Reporting.objects.filter(uuid='bc957403-87af-4473-8ff0-779e3df894ab')

In [ ]:
for a in reps:
    a.form_data['service'] = []
    a.save()

In [ ]:
team=Firm.objects.get(name='RMC_T3_01')

In [ ]:
team

In [ ]:
mdr=MultipleDailyReport.objects.get(number='CART-RDO-2023.18902')

In [ ]:
mdr.firm=team
mdr.save()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

In [ ]:
occs = OccurrenceType.objects.all().values('name')

In [ ]:
occs

In [ ]:
for a in occs:
    a['tamanho_string'] = len(a['name'])

In [ ]:
occs

In [ ]:
df = pd.DataFrame(data=occs)
df

In [ ]:
intervalos=[a for a in range(0,200,10)]
intervalos

In [ ]:
df['group'] = pd.cut(df['tamanho_string'], intervalos)

# Use groupby() para agrupar os dados pela nova coluna
grouped = df.groupby('group')

# Imprima os grupos e suas contagens
last=[]
for name, group in grouped:
    last.append([name,len(group)])
    print(f'{name}: {len(group)}')

In [ ]:
df2 = pd.DataFrame(data=last,columns=['Range de Caracteres','Quantidade Classes por grupo de caracteres'])

In [ ]:
df2

In [ ]:
df2.to_excel('Quantidade Classes por grupo de caracteres.xlsx')

In [ ]:
# Crie uma série Pandas com os dados
s = pd.Series(df2['Quantidade Classes por grupo de caracteres'])

# Crie a função de distribuição acumulada empírica
ecdf = s.value_counts(normalize=True).sort_index().cumsum()

# Crie o gráfico de distribuição acumulada interativo
fig = px.line(x=ecdf.index, y=ecdf.values)

# Defina o título do gráfico e dos eixos x e y
fig.update_layout(title='Distribuição Acumulada', xaxis_title='Quantidade por grupo de caracteres', yaxis_title='Distribuição Acumulativa')

# Defina o intervalo entre os marcadores do eixo x como 10
fig.update_xaxes(dtick=500)

# Transforme o eixo y em porcentagem
fig.update_yaxes(tickformat='.0%')

# fig.add_trace(
#     go.Scatter(
#         mode='markers+text',
#         x=[50,100,200],
#         y=[0.93,0.96,0.99],
#         text=['50-93%','100-96%','200-99%'],
#         textposition="bottom center",
#         marker=dict(
#             color='red',
#             size=5
#         ),
#         showlegend=False
#     )
# )

# Exiba o gráfico
fig.show()

In [ ]:
mdr=MultipleDailyReport.objects.get(uuid='09d4c37f-66f1-4e0a-88aa-482b21d1d471')

In [ ]:
mdr.multiple_daily_report_occurrences.all()[0].__dict__



# Alterações de campos em massa

In [1]:
from helpers.histories import bulk_update_with_history
from django.db.models import Q

In [2]:
company=Company.objects.get(name="CART")

In [28]:
reports = Reporting.objects.filter(Q(form_data__has_key="lenght")| Q(form_data__has_key="width"),company=company, occurrence_type_id__in=[
    '6e69ba68-22cc-4966-a7ff-6a5ce8532374',
    '03c498e9-e1cf-4afd-9e4b-7112a4f2ae87',
    '45b5da54-84e8-4f5d-957b-e7809d5b9b8c',
    '9749758a-53bd-444c-a22d-9d7108a235d7'
])

count = reports.count()
print(count)


906


In [21]:
reports[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fcd9fd3bc10>,
 'uuid': UUID('8029f515-4f82-4a47-894e-e7514a04dca0'),
 'number': '05780.2024',
 'company_id': UUID('96e24ad5-3b23-461f-9300-85c542576894'),
 'road_name': 'SP-270',
 'road_id': 867,
 'km': 417.3526,
 'end_km': 417.3526,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.1,
 'point': <Point object at 0x7fcda19065b0>,
 'direction': '3',
 'lane': '33',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('a539a689-62ea-4280-928d-d0c08c4cb0b5'),
 'firm_id': UUID('2cb6ddc9-a591-4450-9834-2c0d9927bd47'),
 'occurrence_type_id': UUID('03c498e9-e1cf-4afd-9e4b-7112a4f2ae87'),
 'form_data': {'width': 1,
  'height': 1,
  'length': 100,
  'criticality': '2',
  'number_of_persons': '1',
  'final_time_activity': '2024-01-15T20:28:00.000Z',
  'initial_time_activity': '2024-01-15T12:00:00.000Z'},
 'form_metadata': {},
 'executed_at': datetime.datetime(2024, 1, 15, 11, 59, tzinfo=<UTC>),
 'created_at': datet

In [29]:
rep_edit=[]
for rep in reports:
    form_data = rep.form_data

    if 'length' in form_data:
        rep.form_data['executed_length'] = rep.form_data.pop('length')
    
    if 'width' in form_data:
        rep.form_data['executed_width'] = rep.form_data.pop('width')
    
    if 'height' in form_data:
        rep.form_data['executed_height'] = rep.form_data.pop('height')
        
    if 'area' in form_data.keys():
        rep.form_data['executed_area'] = rep.form_data.pop('area')
    elif (rep.form_data.get('executed_length') is not None and rep.form_data.get('executed_width') is not None) and ('area' not in form_data.keys() or 'executed_area' not in form_data.keys()):
        rep.form_data['executed_area'] = rep.form_data.get('executed_length') * rep.form_data.get('executed_width')
    if 'volume' in form_data:
        rep.form_data['executed_volume'] = rep.form_data.pop('volume')
    elif (rep.form_data.get('executed_length') is not None and rep.form_data.get('executed_width') is not None and rep.form_data.get('executed_height') is not None) and ('volume' not in form_data.keys() or 'executed_volume' not in form_data.keys()): 
        rep.form_data['executed_volume'] = rep.form_data.get('executed_length') * rep.form_data.get('executed_width') * rep.form_data.get('executed_height')
        
    rep_edit.append(rep)

In [33]:
rep_edit[2].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fcda1ce3c90>,
 'uuid': UUID('27919ae7-0145-41bb-9ef2-be0a4e7cac1a'),
 'number': '70281.2022',
 'company_id': UUID('96e24ad5-3b23-461f-9300-85c542576894'),
 'road_name': 'SP-270',
 'road_id': 867,
 'km': 608.1,
 'end_km': 608.12,
 'km_reference': 0.0,
 'project_km': 0.0,
 'project_end_km': 0.02,
 'point': <Point object at 0x7fcd9fcf2f40>,
 'direction': '3',
 'lane': '33',
 'track': '',
 'branch': '',
 'address': {},
 'created_by_id': UUID('e47161a0-879d-4a70-9a00-c25ad4e6b64b'),
 'firm_id': UUID('c1e291d9-89f5-4728-805f-68cc8f3eaaf3'),
 'occurrence_type_id': UUID('03c498e9-e1cf-4afd-9e4b-7112a4f2ae87'),
 'form_data': {'city': '19',
  'criticality': '1',
  'type_service': '2',
  'demand_origin': ['11'],
  'contratual_code': 'b.3.2.',
  'number_of_persons': '02',
  'final_time_activity': '2022-11-14T19:30:00.000Z',
  'initial_time_activity': '2022-11-14T18:48:00.000Z',
  'executed_length': 20,
  'executed_width': 6,
  'executed_area': 120

In [34]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=300,user=User.objects.get(username='rlcs'))

# ALTERAÇÕES PELO RDO

In [38]:
from helpers.strings import get_obj_from_path
from helpers.apps.reportings import get_lane

In [39]:
company=Company.objects.get(name="CART")

In [40]:
from openpyxl import load_workbook
filename = "ALTERACOES OCORRENCIAS(1).xlsm"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[0]]
ws

<Worksheet "Planilha3">

In [41]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

{'Serial RDO': 'CART-RDO-2024.00438',
 'Equipe': 'EPA_T1_01',
 'Data': datetime.datetime(2024, 1, 5, 0, 0),
 'Origem': 'Executora',
 'Descrição': 'Preparo para início do trabalho (Separar material, ferramentas, combustível e etc',
 'Horário de Início': datetime.time(6, 30),
 'Horário de Fim': datetime.time(6, 35),
 'Complemento': None,
 'Impacto': datetime.time(0, 5)}

In [42]:
possible_lane_path = (
        "daily_report_occurrence__fields__description__selectoptions__options"
    )
occurrences = get_obj_from_path(
        company.custom_options, possible_lane_path
    )

In [43]:
occurrences

[{'name': 'Acidente sem vítimas', 'value': '0'},
 {'name': 'Deslocamento', 'value': '1'},
 {'name': 'Improdutivo (Justificar)', 'value': '2'},
 {'name': 'Manutenção Veículo/Equip.', 'value': '3'},
 {'name': 'Tempo Impraticável', 'value': '4'},
 {'name': 'Carga e Descarga', 'value': '5'},
 {'name': 'Trabalhando', 'value': '6'},
 {'name': 'Indisponível/Outros', 'value': '7'},
 {'name': 'Preparo para início do trabalho (Separar material, ferramentas, combustível e etc',
  'value': '8'},
 {'name': 'DDS - Diálogo Diário de Segurança', 'value': '9'},
 {'name': 'Deslocamento Ida (Base até o Trecho)', 'value': '10'},
 {'name': 'Preparo para início das atividades no trecho e Check List Preventivo',
  'value': '11'},
 {'name': 'Preparo para almoço', 'value': '12'},
 {'name': 'Horario de almoço', 'value': '13'},
 {'name': 'Preparo Fim das Atividades no Campo (guardar ferramentas',
  'value': '14'},
 {'name': 'Deslocamento Retorno (Trecho até a Base)', 'value': '15'}]

In [44]:
for index,a in tqdm(enumerate(values)):
    rdo=MultipleDailyReport.objects.get(number=a['Serial RDO'], company=company)

0it [00:00, ?it/s]

In [46]:
from tqdm.notebook import tqdm
from datetime import datetime

for index,a in enumerate(tqdm(values)):
    try:
        rdo=MultipleDailyReport.objects.get(number=a['Serial RDO'], company=company)
        firm=rdo.firm
        origin = 'Executora'
        description = next(o['value'] for o in occurrences if o['name'] == a['Descrição'])
        starts_at = a['Horário de Início']
        ends_at = a['Horário de Fim']
        #address_additional = a['Complemento']
        impact_duration = a['Impacto']
        dro = DailyReportOccurrence(firm=firm,description=description,origin=origin,
                                   starts_at=starts_at,ends_at=ends_at,impact_duration=impact_duration)
        
        dro.save()
        rdo.multiple_daily_report_occurrences.add(dro)

    except Exception as e:
        print(f"Erro: {e},linha da planilha:{index+3}")

  0%|          | 0/456 [00:00<?, ?it/s]

In [ ]:
dro

In [ ]:
rdo

In [ ]:
Firm.objects.filter(company__name='Demo Concessionárias',subcompany__name='M&A LTDA')

In [ ]:
occs=OccurrenceType.objects.filter(company__name__startswith='Arteris')

In [ ]:
occs.count()

In [ ]:
occs[0].company.all()[0].name

In [ ]:
for a in occs:
    if a.form_fields['fields'] is not None or a.form_fields['fields'] is not {}:
        for field in a.form_fields['fields']:
            try:
                if 'city' in field.get('apiName'):
                    print(field['dataType'],a.name,next(b.name for b in a.company.all()))
            except:
                pass

In [ ]:
MeasurementBulletin._meta.get_fields()

In [ ]:
mb=MeasurementBulletin.objects.filter(number__in=["AFD-BM-2023.12", "AFD-BM-2023.14", "AFD-BM-2023.13"],
                                     contract__subcompany__company=Company.objects.get(name="Arteris Fernão Dias - Conserva"))

In [ ]:
mb

In [ ]:
mb[0].__dict__

In [ ]:
up=UserPermission.objects.filter(companies__name='CART')

In [ ]:
for a in up:
    print(a.name,a.permissions['MultipleDailyReport'])

In [ ]:
for a in UserInCompany.objects.filter(company__name__icontains='Kapa'):
    print(a.permissions.name,a.user.username)

In [ ]:
company=Company.objects.get(name="CART")

In [ ]:
from openpyxl import load_workbook
filename = "Unidades ativas Kartado Geral (1).xlsx"
wb = load_workbook(filename)
ws = wb[wb.sheetnames[2]]
ws

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)
values[0]

In [ ]:
values[0]['Unidades:'].split(' | ')[0].strip()

In [ ]:
ativas=[]
for a in values:
    ativas.append(Company.objects.get(name=a['Unidades:'].split('|  [')[0].strip()).uuid)
#     Company.objects.get(name=a['Unidades:'].split('|  [')[0].strip())

In [ ]:
Company.objects.filter(uuid__in=ativas).update(active=True)

In [ ]:
Company.objects.all()[0].active

In [ ]:
Company.objects.all().exclude(uuid__in=ativas).update(active=False)

In [ ]:
up=UserPermission.objects.filter(companies__name='CART')

In [ ]:
up.count()

In [ ]:
company=Company.objects.get(name='Sinalizavia (Municípios)')

In [ ]:
for a in company.history.all().order_by('-history_date'):
    print(a.history_date,a.history_user)

In [ ]:
company.history.all().order_by('-history_date')[2].history_user,company.history.all().order_by('-history_date')[2].history_date

In [ ]:
rep=Reporting.objects.get(number='SV-SinV-2023.00010')

In [ ]:
rep.history.all().order_by('-history_date')[0].history_user

In [ ]:
Reporting.history.model.objects.filter(
    history_date__date="2023-07-24", 
    history_type="-", 
    form_data__artesp_code="618034"
)

In [ ]:
occs=OccurrenceType.objects.filter(company__name='CART')

In [ ]:
for a in occs:
    for field in a.form_fields['fields']:
        try:
            if 'altura' in field.get('displayName').lower() and 'executad' in field.get('displayName').lower():
                print(a.name)
        except:
            if 'altura' in field.get('display_name').lower() and 'executad' in field.get('display_name').lower():
                print(a.name)

In [ ]:
for a in occs:
    for field in a.form_fields['fields']:
        try:
            if 'executedHeight' in field.get('apiName'):
                print(a.name)
        except:
            pass

In [ ]:
Reporting.objects.filter(form_data__has_key='executed_height',company__name='CART')

In [ ]:
MultipleDailyReport.objects.get(number='INT-RDO-2023.00134').firm

In [ ]:
con = Contract.objects.get(uuid='f82a9b41-7a23-4f84-8423-42eff51be42f')

In [ ]:
con

In [ ]:
firms_iasin = Firm.objects.filter(subcompany__name='IASIN - SINALIZAÇÕES')
firms_iasin

In [ ]:
rdo=MultipleDailyReport.objects.filter(contract=con,reportings__firm__in=firms_iasin)

In [ ]:
rdo

In [ ]:
mb=MeasurementBulletin.objects.get(number='SV-BM-2023.9')

In [ ]:
mb.delete()

# ARQUIVO SEM LINHAS EM BRANCO

In [28]:
from openpyxl import load_workbook

# Carregar o arquivo Excel
filename = "ALTERACOES OCORRENCIAS(1).xlsm"
wb = load_workbook(filename)

# Selecionar a planilha
ws = wb.active

# Percorrer as linhas da planilha de baixo para cima
for row in reversed(range(1, ws.max_row + 1)):
    # Verificar se todas as células da linha estão vazias
    if all(ws.cell(row=row, column=col).value is None for col in range(1, ws.max_column + 1)):
        # Se todas as células estiverem vazias, remover a linha
        ws.delete_rows(row)

# Salvar as alterações no arquivo
wb.save("seuarquivo_sem_linhas_em_branco.xlsx")
